In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 100*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [3]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240224192712
0	Validation losses: 0.4927, 40.6680, 41.1607	Best Loss: 41.1607 (0)	Accuracy: 81.28%
1	Validation losses: 0.4759, 7.4898, 7.9657	Best Loss: 7.9657 (0)	Accuracy: 83.98%
2	Validation losses: 0.5160, 0.8736, 1.3896	Best Loss: 1.3896 (0)	Accuracy: 81.92%
3	Validation losses: 0.5444, 0.4372, 0.9817	Best Loss: 0.9817 (0)	Accuracy: 80.86%
4	Validation losses: 0.5234, 0.3605, 0.8840	Best Loss: 0.8840 (0)	Accuracy: 81.78%
5	Validation losses: 0.5729, 0.2518, 0.8247	Best Loss: 0.8247 (0)	Accuracy: 78.86%
6	Validation losses: 0.4654, 0.2170, 0.6824	Best Loss: 0.6824 (0)	Accuracy: 83.10%
7	Validation losses: 0.4717, 0.1870, 0.6587	Best Loss: 0.6587 (0)	Accuracy: 83.10%
8	Validation losses: 0.4894, 0.1767, 0.6661	Best Loss: 0.6587 (1)	Accuracy: 82.88%
9	Validation losses: 0.5107, 0.1636, 0.6743	Best Loss: 0.6587 (2)	Accuracy: 81.52%
10	Validation losses: 0.4786, 0.1607, 0.6394	Best Loss: 0.6394 (0)	Accuracy: 83.26%
11	Validation losses: 0.50

29	Validation losses: 2.3041, 1.8841, 4.1883	Best Loss: 0.6203 (17)	Accuracy: 10.08%
30	Validation losses: 2.3037, 1.8824, 4.1861	Best Loss: 0.6203 (18)	Accuracy: 10.08%
31	Validation losses: 2.3028, 1.8812, 4.1840	Best Loss: 0.6203 (19)	Accuracy: 11.12%
32	Validation losses: 2.3039, 1.8796, 4.1836	Best Loss: 0.6203 (20)	Accuracy: 9.14%
Early stopping!
Total running time:	 2710
INFO:tensorflow:Restoring parameters from ./models/model_20240224205014.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    18.5800015926%
RANDOM SEED:   1 	LAMBDA_EXP:   -4.2
Now: 20240224213526
0	Validation losses: 0.4578, 33.0063, 33.4641	Best Loss: 33.4641 (0)	Accuracy: 82.46%
1	Validation losses: 0.4137, 12.1109, 12.5245	Best Loss: 12.5245 (0)	Accuracy: 85.84%
2	Validation losses: 0.4529, 2.3131, 2.7660	Best Loss: 2.7660 (0)	Accuracy: 83.88%
3	Validation losses: 0.5084, 0.5454, 1.0539	Best Loss: 1.0539 (0)	Accu

18	Validation losses: 0.4979, 0.1636, 0.6615	Best Loss: 0.5824 (7)	Accuracy: 83.80%
19	Validation losses: 0.8276, 0.2300, 1.0577	Best Loss: 0.5824 (8)	Accuracy: 65.36%
20	Validation losses: 0.8453, 0.3073, 1.1525	Best Loss: 0.5824 (9)	Accuracy: 69.78%
21	Validation losses: 0.6293, 0.2626, 0.8919	Best Loss: 0.5824 (10)	Accuracy: 78.60%
22	Validation losses: 0.5807, 0.2381, 0.8188	Best Loss: 0.5824 (11)	Accuracy: 82.20%
23	Validation losses: 1.8165, 0.3779, 2.1944	Best Loss: 0.5824 (12)	Accuracy: 19.50%
24	Validation losses: 2.3044, 0.5556, 2.8600	Best Loss: 0.5824 (13)	Accuracy: 9.88%
25	Validation losses: 2.3053, 0.5695, 2.8749	Best Loss: 0.5824 (14)	Accuracy: 9.86%
26	Validation losses: 2.3057, 0.5511, 2.8568	Best Loss: 0.5824 (15)	Accuracy: 9.14%
27	Validation losses: 2.3027, 0.5335, 2.8363	Best Loss: 0.5824 (16)	Accuracy: 9.76%
28	Validation losses: 2.3044, 0.5244, 2.8288	Best Loss: 0.5824 (17)	Accuracy: 9.80%
29	Validation losses: 2.3041, 0.5177, 2.8218	Best Loss: 0.5824 (18)	Accur

37	Validation losses: 0.3856, 0.1110, 0.4965	Best Loss: 0.4965 (0)	Accuracy: 86.72%
38	Validation losses: 0.3838, 0.1105, 0.4942	Best Loss: 0.4942 (0)	Accuracy: 86.62%
39	Validation losses: 0.3782, 0.1102, 0.4884	Best Loss: 0.4884 (0)	Accuracy: 87.04%
40	Validation losses: 0.3771, 0.1098, 0.4869	Best Loss: 0.4869 (0)	Accuracy: 86.92%
41	Validation losses: 0.3766, 0.1097, 0.4862	Best Loss: 0.4862 (0)	Accuracy: 86.96%
42	Validation losses: 0.3779, 0.1097, 0.4875	Best Loss: 0.4862 (1)	Accuracy: 86.62%
43	Validation losses: 0.3813, 0.1099, 0.4912	Best Loss: 0.4862 (2)	Accuracy: 86.64%
44	Validation losses: 0.3796, 0.1098, 0.4893	Best Loss: 0.4862 (3)	Accuracy: 86.54%
45	Validation losses: 0.3880, 0.1099, 0.4979	Best Loss: 0.4862 (4)	Accuracy: 86.58%
46	Validation losses: 0.3837, 0.1103, 0.4941	Best Loss: 0.4862 (5)	Accuracy: 86.00%
47	Validation losses: 0.3891, 0.1101, 0.4992	Best Loss: 0.4862 (6)	Accuracy: 86.32%
48	Validation losses: 0.3767, 0.1104, 0.4871	Best Loss: 0.4862 (7)	Accuracy:

Batches remaining:     1
Test error rate:    16.8400001526%
RANDOM SEED:   2 	LAMBDA_EXP:   -4.6
Now: 20240225033541
0	Validation losses: 0.4054, 16.7613, 17.1666	Best Loss: 17.1666 (0)	Accuracy: 85.68%
1	Validation losses: 0.3747, 11.7510, 12.1257	Best Loss: 12.1257 (0)	Accuracy: 86.28%
2	Validation losses: 0.3910, 6.4195, 6.8105	Best Loss: 6.8105 (0)	Accuracy: 86.08%
3	Validation losses: 0.4963, 2.6777, 3.1741	Best Loss: 3.1741 (0)	Accuracy: 81.54%
4	Validation losses: 0.3875, 0.9492, 1.3367	Best Loss: 1.3367 (0)	Accuracy: 86.20%
5	Validation losses: 0.4364, 0.3993, 0.8357	Best Loss: 0.8357 (0)	Accuracy: 83.54%
6	Validation losses: 0.4300, 0.2624, 0.6925	Best Loss: 0.6925 (0)	Accuracy: 84.60%
7	Validation losses: 0.4472, 0.2216, 0.6687	Best Loss: 0.6687 (0)	Accuracy: 84.46%
8	Validation losses: 0.4051, 0.1974, 0.6025	Best Loss: 0.6025 (0)	Accuracy: 84.76%
9	Validation losses: 0.4623, 0.1703, 0.6326	Best Loss: 0.6025 (1)	Accuracy: 84.54%
10	Validation losses: 0.4670, 0.1471, 0.6142	Be

...

14	Validation losses: 0.3014, 0.2586, 0.5600	Best Loss: 0.5600 (0)	Accuracy: 89.18%
15	Validation losses: 0.3222, 0.2532, 0.5754	Best Loss: 0.5600 (1)	Accuracy: 88.30%
16	Validation losses: 0.3051, 0.2481, 0.5532	Best Loss: 0.5532 (0)	Accuracy: 89.36%
17	Validation losses: 0.3295, 0.2427, 0.5722	Best Loss: 0.5532 (1)	Accuracy: 88.26%
18	Validation losses: 0.3398, 0.2371, 0.5769	Best Loss: 0.5532 (2)	Accuracy: 89.28%
19	Validation losses: 0.3471, 0.2318, 0.5790	Best Loss: 0.5532 (3)	Accuracy: 88.68%
20	Validation losses: 0.3432, 0.2265, 0.5697	Best Loss: 0.5532 (4)	Accuracy: 88.74%
21	Validation losses: 0.3132, 0.2212, 0.5345	Best Loss: 0.5345 (0)	Accuracy: 89.32%
22	Validation losses: 0.3170, 0.2161, 0.5331	Best Loss: 0.5331 (0)	Accuracy: 89.74%
23	Validation losses: 0.3190, 0.2108, 0.5297	Best Loss: 0.5297 (0)	Accuracy: 89.34%
24	Validation losses: 0.3372, 0.2055, 0.5427	Best Loss: 0.5297 (1)	Accuracy: 89.86%
25	Validation losses: 0.3250, 0.2009, 0.5260	Best Loss: 0.5260 (0)	Accuracy:

12	Validation losses: 0.3340, 0.2691, 0.6032	Best Loss: 0.5660 (4)	Accuracy: 87.78%
13	Validation losses: 0.3355, 0.2638, 0.5993	Best Loss: 0.5660 (5)	Accuracy: 88.02%
14	Validation losses: 0.3085, 0.2585, 0.5670	Best Loss: 0.5660 (6)	Accuracy: 88.56%
15	Validation losses: 0.3233, 0.2535, 0.5768	Best Loss: 0.5660 (7)	Accuracy: 88.96%
16	Validation losses: 0.3502, 0.2483, 0.5985	Best Loss: 0.5660 (8)	Accuracy: 88.32%
17	Validation losses: 0.2997, 0.2428, 0.5425	Best Loss: 0.5425 (0)	Accuracy: 89.60%
18	Validation losses: 0.3396, 0.2374, 0.5770	Best Loss: 0.5425 (1)	Accuracy: 88.10%
19	Validation losses: 0.3332, 0.2321, 0.5653	Best Loss: 0.5425 (2)	Accuracy: 88.90%
20	Validation losses: 0.3602, 0.2271, 0.5873	Best Loss: 0.5425 (3)	Accuracy: 87.38%
21	Validation losses: 0.3650, 0.2216, 0.5865	Best Loss: 0.5425 (4)	Accuracy: 88.14%
22	Validation losses: 0.3530, 0.2161, 0.5691	Best Loss: 0.5425 (5)	Accuracy: 88.74%
23	Validation losses: 0.3231, 0.2111, 0.5342	Best Loss: 0.5342 (0)	Accuracy:

19	Validation losses: 0.3110, 0.1764, 0.4873	Best Loss: 0.4810 (2)	Accuracy: 89.32%
20	Validation losses: 0.3198, 0.1752, 0.4950	Best Loss: 0.4810 (3)	Accuracy: 88.92%
21	Validation losses: 0.3301, 0.1737, 0.5037	Best Loss: 0.4810 (4)	Accuracy: 89.50%
22	Validation losses: 0.3318, 0.1716, 0.5035	Best Loss: 0.4810 (5)	Accuracy: 89.04%
23	Validation losses: 0.3750, 0.1698, 0.5447	Best Loss: 0.4810 (6)	Accuracy: 88.82%
24	Validation losses: 0.3398, 0.1679, 0.5077	Best Loss: 0.4810 (7)	Accuracy: 89.86%
25	Validation losses: 0.3209, 0.1660, 0.4869	Best Loss: 0.4810 (8)	Accuracy: 89.88%
26	Validation losses: 0.3324, 0.1640, 0.4963	Best Loss: 0.4810 (9)	Accuracy: 90.04%
27	Validation losses: 0.3415, 0.1620, 0.5035	Best Loss: 0.4810 (10)	Accuracy: 90.00%
28	Validation losses: 0.3821, 0.1602, 0.5423	Best Loss: 0.4810 (11)	Accuracy: 90.14%
29	Validation losses: 0.3689, 0.1584, 0.5273	Best Loss: 0.4810 (12)	Accuracy: 90.00%
30	Validation losses: 0.3993, 0.1567, 0.5561	Best Loss: 0.4810 (13)	Accur

11	Validation losses: 0.3418, 0.1235, 0.4653	Best Loss: 0.4326 (3)	Accuracy: 87.80%
12	Validation losses: 0.3549, 0.1236, 0.4786	Best Loss: 0.4326 (4)	Accuracy: 88.20%
13	Validation losses: 0.3342, 0.1237, 0.4579	Best Loss: 0.4326 (5)	Accuracy: 88.90%
14	Validation losses: 0.3336, 0.1239, 0.4575	Best Loss: 0.4326 (6)	Accuracy: 88.08%
15	Validation losses: 0.3410, 0.1241, 0.4651	Best Loss: 0.4326 (7)	Accuracy: 88.38%
16	Validation losses: 0.3185, 0.1242, 0.4428	Best Loss: 0.4326 (8)	Accuracy: 88.84%
17	Validation losses: 0.3286, 0.1244, 0.4530	Best Loss: 0.4326 (9)	Accuracy: 89.02%
18	Validation losses: 0.3570, 0.1247, 0.4818	Best Loss: 0.4326 (10)	Accuracy: 88.96%
19	Validation losses: 0.3265, 0.1253, 0.4517	Best Loss: 0.4326 (11)	Accuracy: 89.40%
20	Validation losses: 0.3569, 0.1255, 0.4823	Best Loss: 0.4326 (12)	Accuracy: 88.36%
21	Validation losses: 0.3196, 0.1254, 0.4450	Best Loss: 0.4326 (13)	Accuracy: 89.54%
22	Validation losses: 0.3441, 0.1252, 0.4692	Best Loss: 0.4326 (14)	Accu

4	Validation losses: 0.3254, 0.0780, 0.4035	Best Loss: 0.4035 (0)	Accuracy: 88.42%
5	Validation losses: 0.3412, 0.0783, 0.4195	Best Loss: 0.4035 (1)	Accuracy: 87.38%
6	Validation losses: 0.3044, 0.0785, 0.3829	Best Loss: 0.3829 (0)	Accuracy: 88.28%
7	Validation losses: 0.3208, 0.0788, 0.3996	Best Loss: 0.3829 (1)	Accuracy: 88.36%
8	Validation losses: 0.3052, 0.0791, 0.3844	Best Loss: 0.3829 (2)	Accuracy: 88.98%
9	Validation losses: 0.3396, 0.0794, 0.4190	Best Loss: 0.3829 (3)	Accuracy: 88.40%
10	Validation losses: 0.3337, 0.0798, 0.4135	Best Loss: 0.3829 (4)	Accuracy: 88.24%
11	Validation losses: 0.3277, 0.0802, 0.4079	Best Loss: 0.3829 (5)	Accuracy: 88.70%
12	Validation losses: 0.3202, 0.0806, 0.4008	Best Loss: 0.3829 (6)	Accuracy: 89.02%
13	Validation losses: 0.3511, 0.0811, 0.4322	Best Loss: 0.3829 (7)	Accuracy: 87.80%
14	Validation losses: 0.3500, 0.0816, 0.4316	Best Loss: 0.3829 (8)	Accuracy: 87.70%
15	Validation losses: 0.3114, 0.0822, 0.3936	Best Loss: 0.3829 (9)	Accuracy: 89.12